# LSTM Volatility Forecasting with Hyperparameter Optimization

## Overview
This notebook builds and optimizes a Long Short-Term Memory (LSTM) neural network to predict 10-day ahead volatility in financial markets. The project includes:

- **Data preparation**: Feature engineering with lagged volatility values
- **Hyperparameter tuning**: Automated search using Keras Tuner's Hyperband algorithm to find optimal model architecture, regularization, and training parameters
- **Model evaluation**: Comprehensive performance analysis including:
  - Regression metrics (R², MSE, MAE)
  - Directional accuracy (predicting volatility increases/decreases)
  - Visual diagnostics across 12+ detailed plots
- **Comparison**: Benchmarking the optimized model against a baseline LSTM

The goal is to forecast future volatility levels with high accuracy while understanding the model's ability to predict directional changes—critical for risk management and trading applications.

---

## Dependencies

Core libraries for data processing, machine learning, and visualization:
- **pandas/numpy**: Data manipulation and numerical operations
- **scikit-learn**: Data preprocessing (StandardScaler), baseline models (LinearRegression), and evaluation metrics
- **matplotlib/seaborn**: Visualization and plotting
- **tensorflow/keras**: Deep learning framework for building and training LSTM models
  - Sequential: Model architecture
  - LSTM, Dense, Dropout, BatchNormalization: Neural network layers
  - EarlyStopping: Training callbacks to prevent overfitting
  - regularizers: L1/L2 regularization techniques
- **keras-tuner**: Automated hyperparameter optimization using Hyperband algorithm
- **scipy.stats**: Statistical analysis and distribution testing
- **os/pathlib**: File system operations and path management

In [1]:
### Dependencies
import pandas as pd

import numpy as np

from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, confusion_matrix, classification_report, accuracy_score

import matplotlib.pyplot as plt

import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
import keras_tuner as kt

from scipy import stats

import os

2025-11-19 09:38:10.473980: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-19 09:38:10.475053: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 09:38:10.492456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-19 09:38:10.492471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-19 09:38:10.492979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

Using TensorFlow backend


In [9]:
### Configure TF to use 16 threads of my CPU (optional)
#tf.config.threading.set_intra_op_parallelism_threads(16)

# Set random seeds for reproducibility
#def set_seeds(seed=42):
#    np.random.seed(seed)
#    tf.random.set_seed(seed)
#    random.seed(seed)
#    os.environ['PYTHONHASHSEED'] = str(seed)
#    
#    # Make TensorFlow operations deterministic
#    os.environ['TF_DETERMINISTIC_OPS'] = '1'
#    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
#    
#    # Configure TensorFlow for reproducibility
#    tf.config.experimental.enable_op_determinism()
#
## Call at the start
#set_seeds(42)

## Data Loading and Feature Engineering

### Purpose
Load historical S&P 500 data and engineer features suitable for time-series volatility forecasting with LSTM models.

### Key Functions

**`load_csv()`**
- Loads CSV data from the `data/` directory
- Parses dates in MM/DD/YY format
- Corrects century ambiguity (e.g., '30' → 1930, not 2030)
- Sorts chronologically for time-series analysis

**`add_features()`**
- **Returns & Volatility**: Daily returns and rolling 10-day volatility (our target metric)
- **Price Features**: Intraday ranges, close-to-open changes, moving averages (5-day, 10-day)
- **Volume Features**: Volume percent change and 5-day moving average
- **Lag Features**: Previous day returns and volatility for temporal context
- **Target Variable**: 10-day ahead volatility (`Volatility_future`)
- **Data Cleaning**: 
  - Replaces infinite values with NaN
  - Fills missing volume data
  - Removes rows with missing key features
- **Standardization**: Scales all features using StandardScaler for optimal neural network training

### Data Split
- **80% training set**: Used for model training and hyperparameter tuning (with internal validation split)
- **20% test set**: Held out for final evaluation, maintains temporal order to simulate real-world forecasting

This approach ensures the model learns from historical patterns while being evaluated on unseen future data.

In [3]:
### Data Loading

def load_csv(file_name: str) -> pd.DataFrame:
    """
    Load a CSV file from the data directory.
    """
    data_path = Path.cwd() / "data" / file_name 
    if not data_path.exists():
        raise FileNotFoundError(f"{data_path} not found.")

    df = pd.read_csv(data_path, parse_dates=['Date'])
    
    # Parse 'Date' column explicitly
    # MM/DD/YY format
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y', errors='coerce')
    
    # Correct future years (pandas may interpret '30' as 2030)
    # Assume dates from 1928–2020
    df.loc[df['Date'] > pd.Timestamp.today(), 'Date'] -= pd.offsets.DateOffset(years=100)
    
    # Sort by date
    df = df.sort_values('Date').reset_index(drop=True)
    return df 



def add_features(df: pd.DataFrame, rolling_window: int = 10):
    """
    Add features for ML: returns, rolling volatility, price/volume features, lags.
    Handles NaN and infinite values, ready for scaling.
    """
    df = df.copy()
    
    # Returns and volatility
    df['Return'] = df['Adj Close'].pct_change()
    df['Volatility'] = df['Return'].rolling(rolling_window).std()

    # Price-based features
    df['High_Low_pct'] = (df['High'] - df['Low']) / df['Close']
    df['Close_Open_pct'] = (df['Close'] - df['Open']) / df['Open']
    df['MA5'] = df['Close'].rolling(5).mean()
    df['MA10'] = df['Close'].rolling(10).mean()

    # Volume-based features
    df['Volume_pct_change'] = df['Volume'].pct_change()
    df['Volume_MA5'] = df['Volume'].rolling(5).mean()

    # Lag features
    df['Return_lag1'] = df['Return'].shift(1)
    df['Return_lag2'] = df['Return'].shift(2)
    df['Volatility_lag1'] = df['Volatility'].shift(1)

    # Target: 10 days-ahead volatility
    df['Volatility_future'] = df['Volatility'].shift(-10)

    # Replace inf with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Fill volume-related NaNs (common in early data)
    df['Volume_pct_change'] = df['Volume_pct_change'].fillna(0)
    df['Volume_MA5'] = df['Volume_MA5'].ffill()

    # Drop only rows where the key features are missing
    df = df[df['Volatility'].notna() & df['Volatility_future'].notna()]
    df.reset_index(drop=True, inplace=True)

    # Features to scale
    features = [
        'Return', 'High_Low_pct', 'Close_Open_pct', 'MA5', 'MA10',
        'Volume_pct_change', 'Volume_MA5', 'Return_lag1', 'Return_lag2', 'Volatility_lag1'
    ]

    # Scaling
    scaler = StandardScaler()
    df[features] = scaler.fit_transform(df[features].values)

    target_col = 'Volatility_future'
    return df, features, target_col


# Load raw data
df = load_csv("SPX.csv")

# Preprocess
df_prepared, feature_cols, target_col = add_features(df)

X = df_prepared[feature_cols]
y = df_prepared[target_col]

# Drop any row where X or y has NaN
mask = (~X.isna().any(axis=1)) & (~y.isna())
X = X[mask]
y = y[mask]

print(X.head(), y.head())


# Split data into train (80%) and test (20%) by time
split_idx = int(len(df_prepared) * 0.8)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

/tmp/ipykernel_1097440/1616189166.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(data_path, parse_dates=['Date'])


     Return  High_Low_pct  Close_Open_pct       MA5      MA10  \
1  0.023311      -0.92304       -0.023226 -0.647124 -0.647122   
2 -0.217569      -0.92304       -0.023226 -0.647148 -0.647185   
3  0.554534      -0.92304       -0.023226 -0.647173 -0.647209   
4  0.454640      -0.92304       -0.023226 -0.647201 -0.647234   
5  0.737950      -0.92304       -0.023226 -0.647104 -0.647214   

   Volume_pct_change  Volume_MA5  Return_lag1  Return_lag2  Volatility_lag1  
1          -0.060194   -0.522899    -1.399673     0.499832        -0.166255  
2          -0.060194   -0.522899     0.023281    -1.399676        -0.217933  
3          -0.060194   -0.522899    -0.217598     0.023273        -0.217969  
4          -0.060194   -0.522899     0.554501    -0.217605        -0.203854  
5          -0.060194   -0.522899     0.454607     0.554491        -0.210628   1    0.007275
2    0.007273
3    0.007181
4    0.008303
5    0.007715
Name: Volatility_future, dtype: float64
Train size: 18642, Test size: 4

## Hyperparameter Optimization with Keras Tuner

### Purpose
Automatically search for the optimal LSTM architecture and training configuration using the Hyperband algorithm—a smart resource allocation method that efficiently explores the hyperparameter space.

### Model Building Function (`build_model`)

The function defines a flexible LSTM architecture where Keras Tuner can optimize:

**Architecture Parameters:**
- **LSTM Units**: 32-256 neurons in first layer (step: 32)
- **Second LSTM Layer**: Optional stacked LSTM (16-128 units)
- **Dense Layer**: Optional fully-connected layer before output (16-64 units)
- **Batch Normalization**: Optional after LSTM layers for training stability

**Regularization (Prevent Overfitting):**
- **Dropout**: 0.0-0.5 (randomly drops neurons during training)
- **Recurrent Dropout**: 0.0-0.3 (dropout on LSTM recurrent connections)
- **Weight Regularization**: None, L1, L2, or L1+L2 (elastic net)
  - L1: Promotes sparsity (some weights → 0)
  - L2: Prevents large weights (most common)
  - Strength: 10⁻⁵ to 10⁻² (log scale)

**Training Parameters:**
- **Optimizer**: Adam, RMSprop, or SGD (with momentum if SGD)
- **Learning Rate**: 10⁻⁴ to 10⁻² (log scale)
- **Loss Function**: 
  - MSE (penalizes large errors heavily)
  - MAE (robust to outliers)
  - Huber (combines MSE + MAE)
  - LogCosh (smooth MAE approximation)

### Hyperband Algorithm

**How it works:**
1. Trains many configurations with few epochs
2. Eliminates worst performers (keeps top 1/3)
3. Gives survivors more training epochs
4. Repeats until finding the best model

**Configuration:**
- `max_epochs=50`: Maximum training epochs for top candidates
- `factor=3`: Keeps top 1/3 each round (aggressive pruning)
- `validation_split=0.2`: Uses 20% of training data for validation

**Callbacks:**
- **EarlyStopping**: Stops if validation loss doesn't improve for 7 epochs
- **ReduceLROnPlateau**: Cuts learning rate by half if stuck for 3 epochs

### Why This Matters

Instead of manually testing hundreds of configurations (which could take hours), Hyperband intelligently allocates compute time—spending more resources on promising models and quickly discarding poor ones. This typically finds near-optimal hyperparameters in hours rather than days.

### Output

The search returns the best hyperparameter configuration, which is then used to train the final model on the full training set.

In [4]:
### Define the model building function
def build_model(hp):
    model = Sequential()
    
    # Tune number of LSTM units
    units = hp.Int('units', min_value=32, max_value=256, step=32)
    
    # Tune dropout rates
    dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    recurrent_dropout = hp.Float('recurrent_dropout', min_value=0.0, max_value=0.3, step=0.1)
    
    # Tune regularization type and strength
    reg_type = hp.Choice('regularization_type', ['none', 'l1', 'l2', 'l1_l2'])
    reg_strength = hp.Float('reg_strength', min_value=1e-5, max_value=1e-2, sampling='log')
    
    # Create regularizer based on choice
    if reg_type == 'l1':
        kernel_reg = regularizers.l1(reg_strength)
        recurrent_reg = regularizers.l1(reg_strength * 0.1)  # Lighter on recurrent
    elif reg_type == 'l2':
        kernel_reg = regularizers.l2(reg_strength)
        recurrent_reg = regularizers.l2(reg_strength * 0.1)
    elif reg_type == 'l1_l2':
        kernel_reg = regularizers.l1_l2(l1=reg_strength, l2=reg_strength)
        recurrent_reg = regularizers.l1_l2(l1=reg_strength * 0.1, l2=reg_strength * 0.1)
    else:
        kernel_reg = None
        recurrent_reg = None
    
    # Decide if we need a second LSTM layer first
    add_second_layer = hp.Boolean('add_second_layer')
    
    # First LSTM layer with regularization
    # Must return sequences if adding another LSTM layer
    model.add(LSTM(
        units=units,
        input_shape=(X_train.shape[1], 1),
        dropout=dropout,
        recurrent_dropout=recurrent_dropout,
        kernel_regularizer=kernel_reg,
        recurrent_regularizer=recurrent_reg,
        return_sequences=add_second_layer  # True only if adding second layer
    ))
    
    # Optionally add batch normalization after first LSTM
    if hp.Boolean('use_batch_norm_1'):
        model.add(BatchNormalization())
    
    # Optionally add a second LSTM layer
    if add_second_layer:
        units_2 = hp.Int('units_2', min_value=16, max_value=128, step=16)
        model.add(LSTM(
            units=units_2,
            dropout=dropout,
            recurrent_dropout=recurrent_dropout,
            kernel_regularizer=kernel_reg,
            recurrent_regularizer=recurrent_reg,
            return_sequences=False  # Final LSTM layer
        ))
        
        if hp.Boolean('use_batch_norm_2'):
            model.add(BatchNormalization())
    
    # Optionally add dense layer before output
    if hp.Boolean('add_dense_layer'):
        dense_units = hp.Int('dense_units', min_value=16, max_value=64, step=16)
        model.add(Dense(
            dense_units, 
            activation='relu',
            kernel_regularizer=kernel_reg
        ))
        model.add(Dropout(dropout))
    
    # Output layer
    model.add(Dense(1))
    
    # Tune learning rate
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    # Tune optimizer
    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd'])
    
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:  # sgd
        momentum = hp.Float('momentum', 0.0, 0.9, step=0.1)
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    
    # Tune loss function
    loss_choice = hp.Choice('loss_function', ['mse', 'mae', 'huber', 'logcosh'])
    
    if loss_choice == 'huber':
        loss = tf.keras.losses.Huber(delta=hp.Float('huber_delta', 0.5, 2.0, step=0.5))
    else:
        loss = loss_choice
    
    # Compile model
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['mae', 'mse']
    )
    
    return model

# Create the tuner (using Hyperband algorithm)
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=50,
    factor=3,
    directory='lstm_tuning_advanced',
    project_name='hyperparameter_search_regularized',
    overwrite=True
)

# Print search space summary
print("Search Space Summary:")
print(tuner.search_space_summary())

# Prepare data
X_train_reshaped = X_train.values.reshape(-1, X_train.shape[1], 1)
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1)

# Enhanced callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    restore_best_weights=True,
    min_delta=1e-6
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=0
)

# Run the hyperparameter search
print("\nStarting hyperparameter search...")
tuner.search(
    X_train_reshaped, 
    y_train.values,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n" + "="*70)
print("BEST HYPERPARAMETERS FOUND:")
print("="*70)
print(f"LSTM Units: {best_hps.get('units')}")
print(f"Dropout Rate: {best_hps.get('dropout'):.2f}")
print(f"Recurrent Dropout: {best_hps.get('recurrent_dropout'):.2f}")
print(f"Regularization Type: {best_hps.get('regularization_type')}")
print(f"Regularization Strength: {best_hps.get('reg_strength'):.6f}")
print(f"Optimizer: {best_hps.get('optimizer')}")
if best_hps.get('optimizer') == 'sgd':
    print(f"  Momentum: {best_hps.get('momentum'):.2f}")
print(f"Learning Rate: {best_hps.get('learning_rate'):.6f}")
print(f"Loss Function: {best_hps.get('loss_function')}")
if best_hps.get('loss_function') == 'huber':
    print(f"  Huber Delta: {best_hps.get('huber_delta'):.2f}")
print(f"Batch Normalization (Layer 1): {best_hps.get('use_batch_norm_1')}")
print(f"Add Second LSTM Layer: {best_hps.get('add_second_layer')}")
if best_hps.get('add_second_layer'):
    print(f"  Second Layer Units: {best_hps.get('units_2')}")
    print(f"  Batch Normalization (Layer 2): {best_hps.get('use_batch_norm_2')}")
print(f"Add Dense Layer: {best_hps.get('add_dense_layer')}")
if best_hps.get('add_dense_layer'):
    print(f"  Dense Layer Units: {best_hps.get('dense_units')}")
print("="*70)

Trial 90 Complete [00h 02m 25s]
val_loss: 0.010515527799725533

Best val_loss So Far: 1.238825552718481e-05
Total elapsed time: 00h 24m 17s

BEST HYPERPARAMETERS FOUND:
LSTM Units: 64
Dropout Rate: 0.20
Recurrent Dropout: 0.00
Regularization Type: none
Regularization Strength: 0.000022
Optimizer: adam
Learning Rate: 0.001075
Loss Function: huber
  Huber Delta: 2.00
Batch Normalization (Layer 1): False
Add Second LSTM Layer: True
  Second Layer Units: 96
  Batch Normalization (Layer 2): False
Add Dense Layer: False


## Training the Optimized Model

### Process

**1. Build Best Model**
- Constructs the LSTM architecture using the optimal hyperparameters found by Keras Tuner
- Alternatively, can load a previously saved model (commented line)

**2. Train on Full Training Set**
- Uses full 80% training data (no longer splitting for hyperparameter validation)
- Validates against the held-out 20% test set to monitor generalization
- Early stopping prevents overfitting (stops if test performance degrades)
- Maximum 50 epochs, but typically stops earlier

**3. Evaluation Metrics**
- **Mean Squared Error (MSE)**: Average squared difference between predicted and actual volatility (lower is better)
- **R² Score**: Proportion of variance explained by the model (0-1 scale, higher is better)
  - R² = 0: Model performs no better than predicting the mean
  - R² = 1: Perfect predictions
  - R² = 0.57 means the model explains 57% of volatility variation

### Baseline Comparison

To demonstrate the value of hyperparameter optimization, we train a simple baseline LSTM:
- **Original Model**: Single 50-unit LSTM layer, default Adam optimizer, MSE loss
- **Tuned Model**: Optimized architecture with regularization, custom loss function, etc.

The comparison shows percentage improvement in both MSE (prediction accuracy) and R² (variance explained).

### Model Persistence

The best model is saved in Keras native format (`.keras`) for future use:
- Faster load times than legacy HDF5 format
- Better compatibility with newer TensorFlow versions
- Includes full architecture, weights, and optimizer state

### Top Candidates

Displays the top 3 hyperparameter configurations found during the search, useful for:
- Understanding what patterns work well (e.g., does adding a second layer consistently help?)
- Ensemble modeling (combining predictions from multiple good models)
- Future experimentation starting points

### Expected Results

Typical improvements from hyperparameter tuning:
- **20-40% reduction in MSE** (more accurate predictions)
- **30-60% improvement in R²** (better variance capture)
- More stable training (less prone to overfitting)

In [5]:
### Build and train the best model
print("\nTraining the best model...")
best_model = tuner.hypermodel.build(best_hps)
#best_model = load_model('best_lstm_model.keras')

history = best_model.fit(
    X_train_reshaped,
    y_train.values,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_reshaped, y_test.values),
    callbacks=[early_stop],
    verbose=1
)

# Predict on the test set
y_pred = best_model.predict(X_test_reshaped)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print("\n" + "="*60)
print("FINAL MODEL PERFORMANCE:")
print("="*60)
print(f"Mean Squared Error: {mse:.6f}")
print(f"R² Score: {r2:.6f}")
print(f"Accuracy (R²): {r2 * 100:.2f}%")
print("="*60)

# Compare with original model
print("\n" + "="*60)
print("COMPARISON WITH ORIGINAL MODEL:")
print("="*60)

# Train original, simple LSTM model for comparison
original_model = Sequential([
    LSTM(50, input_shape=(X_train.shape[1], 1)),
    Dense(1)
])
original_model.compile(optimizer='adam', loss='mse')
original_model.fit(
    X_train_reshaped, 
    y_train.values, 
    epochs=50, 
    batch_size=32, 
    verbose=0
)

y_pred_original = original_model.predict(X_test_reshaped)
mse_original = mean_squared_error(y_test, y_pred_original)
r2_original = r2_score(y_test, y_pred_original)

print(f"Original Model - MSE: {mse_original:.6f}, R²: {r2_original * 100:.2f}%")
print(f"Tuned Model    - MSE: {mse:.6f}, R²: {r2 * 100:.2f}%")
print(f"Improvement    - MSE: {((mse_original - mse) / mse_original * 100):.2f}%")
print(f"Improvement    - R²: {((r2 - r2_original) / abs(r2_original) * 100):.2f}%")
print("="*60)

# Save the best model
best_model.save('best_lstm_model.keras')
print("\nBest model saved as 'best_lstm_model.keras'")

# Show top 3 models
print("\n" + "="*60)
print("TOP 3 MODELS:")
print("="*60)
for i, hps in enumerate(tuner.get_best_hyperparameters(num_trials=3), 1):
    print(f"\nModel {i}:")
    print(f"  Units: {hps.get('units')}, Dropout: {hps.get('dropout'):.2f}")
    print(f"  Learning Rate: {hps.get('learning_rate'):.6f}")
    print(f"  Second Layer: {hps.get('add_second_layer')}")


Training the best model...
Epoch 1/50
583/583 [==============================] - 3s 4ms/step - loss: 2.4361e-05 - mae: 0.0045 - mse: 4.8722e-05 - val_loss: 4.0011e-05 - val_mae: 0.0062 - val_mse: 8.0022e-05
Epoch 2/50
583/583 [==============================] - 2s 3ms/step - loss: 1.7745e-05 - mae: 0.0037 - mse: 3.5489e-05 - val_loss: 1.8304e-05 - val_mae: 0.0039 - val_mse: 3.6608e-05
Epoch 3/50
583/583 [==============================] - 2s 3ms/step - loss: 1.7123e-05 - mae: 0.0036 - mse: 3.4245e-05 - val_loss: 1.7092e-05 - val_mae: 0.0035 - val_mse: 3.4183e-05
Epoch 4/50
583/583 [==============================] - 2s 3ms/step - loss: 1.6462e-05 - mae: 0.0035 - mse: 3.2924e-05 - val_loss: 1.4342e-05 - val_mae: 0.0039 - val_mse: 2.8683e-05
Epoch 5/50
583/583 [==============================] - 2s 3ms/step - loss: 1.6016e-05 - mae: 0.0035 - mse: 3.2031e-05 - val_loss: 1.4678e-05 - val_mae: 0.0038 - val_mse: 2.9356e-05
Epoch 6/50
583/583 [==============================] - 2s 3ms/step - loss

## Model Performance Visualization and Diagnostics

### Purpose
Generate 12 detailed diagnostic plots to thoroughly evaluate the LSTM model's predictive performance from multiple angles. All plots are saved as high-resolution images (300 DPI) in the `figures/` directory.

### Diagnostic Plots Generated

**1. Actual vs Predicted Timeline**
- Time series overlay showing how well predictions track actual volatility
- Shaded area visualizes prediction errors
- Includes key metrics (R², RMSE, MAE) in text box

**2. Scatter Plot: Predicted vs Actual**
- Each point represents one prediction
- Red dashed line = perfect predictions
- Green line = actual fitted relationship
- Correlation coefficient shows linear relationship strength

**3. Residuals Over Time**
- Shows prediction errors (actual - predicted) chronologically
- Helps identify systematic biases or time-dependent patterns
- ±2σ bands indicate 95% expected error range
- Clustering outside bands suggests model struggles in certain periods

**4. Residuals Distribution**
- Histogram of prediction errors with normal curve overlay
- **Skewness**: Measures asymmetry (0 = symmetric)
  - Positive: More large overestimations
  - Negative: More large underestimations
- **Kurtosis**: Measures tail heaviness (0 = normal)
  - Positive: More extreme errors than expected
  - Negative: Fewer extreme errors

**5. Q-Q Plot (Quantile-Quantile)**
- Tests if residuals follow normal distribution
- Points on diagonal = normally distributed errors (ideal for regression)
- Deviations indicate non-normal error patterns

**6. Absolute Error Timeline**
- Magnitude of errors over time (ignoring direction)
- Rolling 50-period average smooths short-term noise
- Identifies periods where model struggles most

**7. Percentage Error Distribution**
- Errors expressed as % of actual volatility
- MAPE (Mean Absolute Percentage Error) = average % error
- More interpretable than absolute errors for stakeholders

**8. Error vs Volatility Level**
- Scatter showing if errors increase with volatility magnitude
- Positive trend = model struggles more in high volatility periods
- Flat trend = consistent accuracy across volatility regimes

**9. Accuracy by Volatility Quartile**
- Splits test data into 4 groups (low, med-low, med-high, high volatility)
- Shows MAE and R² for each quartile
- Identifies which market conditions model handles best/worst

**10. Cumulative Absolute Error**
- Running sum of all errors over time
- Steady linear growth = consistent error rate
- Accelerating growth = model degrading over time
- Useful for assessing long-term reliability

**11. Error Autocorrelation**
- Tests if errors are independent across time
- Significant autocorrelation = errors are predictable (model missing patterns)
- Random pattern = model captured all learnable structure

**12. Predictions with Confidence Intervals**
- Shows predictions with ±2σ uncertainty bands
- Wider bands = less confidence
- Helps quantify prediction reliability for decision-making

### Statistical Summary

Comprehensive metrics printed to console:
- **Regression Metrics**: R², MSE, RMSE, MAE, MAPE
- **Residual Statistics**: Mean, median, std dev, min/max, skewness, kurtosis
- **Error Distribution**: % of predictions within 1 MAE, 2 MAE, 1σ, 2σ
- **Quartile Analysis**: Performance breakdown by volatility level

### Interpretation Guidelines

**Good Model Indicators:**
- High R² (>0.5 for financial data is strong)
- Residuals centered near zero (mean ≈ 0)
- Normally distributed errors (Q-Q plot linear)
- No autocorrelation in residuals
- Consistent accuracy across volatility quartiles

**Warning Signs:**
- Systematic bias (residual mean far from 0)
- Heavy tails (kurtosis >> 0) = unexpected large errors
- Strong autocorrelation = missing temporal patterns
- Degrading cumulative error = model unstable over time

In [7]:
# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Dates from dataframe:
test_dates = df_prepared['Date'].iloc[split_idx:-1].values

# Calculate metrics
y_pred = y_pred.flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Calculate residuals
residuals = y_test - y_pred.flatten()
residuals_pct = (residuals / y_test) * 100

print("="*70)
print("GENERATING INDIVIDUAL PLOTS...")
print("="*70)

# ============================================================================
# Plot 1: Actual vs Predicted Over Time (Main Plot)
# ============================================================================
print("\n1. Creating Actual vs Predicted timeline...")
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(test_dates, y_test, label='Actual Volatility', linewidth=2, alpha=0.8, color='#2E86AB')
ax.plot(test_dates, y_pred, label='Predicted Volatility', linewidth=2, alpha=0.8, 
        color='#A23B72', linestyle='--')
ax.fill_between(test_dates, y_test, y_pred, alpha=0.2, color='gray', label='Prediction Error')
ax.set_title('Actual vs Predicted 10-Day Ahead Volatility', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Volatility', fontsize=12)
ax.legend(loc='upper left', fontsize=11, framealpha=0.9)
ax.grid(True, alpha=0.3)

# Add text box with metrics
textstr = f'R² = {r2:.4f}\nRMSE = {rmse:.6f}\nMAE = {mae:.6f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', horizontalalignment='right', bbox=props)

ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.savefig('figures/01_actual_vs_predicted_timeline.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 2: Scatter Plot - Actual vs Predicted
# ============================================================================
print("2. Creating scatter plot...")
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(y_test, y_pred, alpha=0.5, s=30, edgecolors='k', linewidth=0.5)

# Add perfect prediction line
min_val = min(y_test.min(), y_pred.min())
max_val = max(y_test.max(), y_pred.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2.5, label='Perfect Prediction')

# Add regression line
z = np.polyfit(y_test, y_pred.flatten(), 1)
p = np.poly1d(z)
ax.plot(y_test, p(y_test), "g-", alpha=0.8, linewidth=2.5, label=f'Fit: y={z[0]:.3f}x+{z[1]:.6f}')

ax.set_xlabel('Actual Volatility', fontsize=12)
ax.set_ylabel('Predicted Volatility', fontsize=12)
ax.set_title('Predicted vs Actual Scatter Plot', fontsize=16, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Add correlation coefficient
corr = np.corrcoef(y_test, y_pred.flatten())[0, 1]
ax.text(0.05, 0.95, f'Correlation: {corr:.4f}', transform=ax.transAxes,
        fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

plt.tight_layout()
plt.savefig('figures/02_scatter_actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 3: Residuals Over Time
# ============================================================================
print("3. Creating residuals timeline...")
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(test_dates, residuals, linewidth=1, alpha=0.7, color='darkred')
ax.axhline(y=0, color='black', linestyle='-', linewidth=1.5)
ax.axhline(y=residuals.mean(), color='blue', linestyle='--', linewidth=1.5, 
           label=f'Mean: {residuals.mean():.6f}')
ax.fill_between(test_dates, 0, residuals, alpha=0.3, color='darkred')

# Add confidence bands (±2 std)
std_residuals = residuals.std()
ax.axhline(y=2*std_residuals, color='orange', linestyle=':', linewidth=1.5, alpha=0.7, label='±2σ')
ax.axhline(y=-2*std_residuals, color='orange', linestyle=':', linewidth=1.5, alpha=0.7)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Residuals', fontsize=12)
ax.set_title('Prediction Errors Over Time', fontsize=16, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('figures/03_residuals_timeline.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 4: Residuals Distribution
# ============================================================================
print("4. Creating residuals distribution...")
fig, ax = plt.subplots(figsize=(10, 7))
ax.hist(residuals, bins=50, density=True, alpha=0.7, color='skyblue', edgecolor='black')

# Overlay normal distribution
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
ax.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2.5, label='Normal Distribution')

ax.set_xlabel('Residuals', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Residuals Distribution', fontsize=16, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add skewness and kurtosis
skew = stats.skew(residuals)
kurt = stats.kurtosis(residuals)
textstr = f'Skewness: {skew:.3f}\nKurtosis: {kurt:.3f}\nMean: {mu:.6f}\nStd Dev: {sigma:.6f}'
ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig('figures/04_residuals_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 5: Q-Q Plot
# ============================================================================
print("5. Creating Q-Q plot...")
fig, ax = plt.subplots(figsize=(10, 8))
stats.probplot(residuals, dist="norm", plot=ax)
ax.set_title('Q-Q Plot (Normality Check)', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.get_lines()[0].set_markerfacecolor('skyblue')
ax.get_lines()[0].set_markeredgecolor('black')
ax.get_lines()[0].set_markersize(6)

plt.tight_layout()
plt.savefig('figures/05_qq_plot.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 6: Absolute Error Over Time
# ============================================================================
print("6. Creating absolute error timeline...")
fig, ax = plt.subplots(figsize=(14, 6))
abs_errors = np.abs(residuals)
ax.plot(test_dates, abs_errors, linewidth=1, alpha=0.7, color='purple', label='Absolute Error')
ax.axhline(y=mae, color='red', linestyle='--', linewidth=2, label=f'MAE: {mae:.6f}')

# Add rolling mean of absolute errors
window = 50
rolling_mae = pd.Series(abs_errors).rolling(window=window, min_periods=1).mean()
ax.plot(test_dates, rolling_mae, linewidth=2.5, alpha=0.8, color='orange',
        label=f'Rolling MAE (window={window})')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Absolute Error', fontsize=12)
ax.set_title('Absolute Prediction Error Over Time', fontsize=16, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('figures/06_absolute_error_timeline.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 7: Percentage Error Distribution
# ============================================================================
print("7. Creating percentage error distribution...")
fig, ax = plt.subplots(figsize=(10, 7))
# Remove outliers for better visualization
pct_errors_clean = residuals_pct[np.abs(residuals_pct) < np.percentile(np.abs(residuals_pct), 95)]

ax.hist(pct_errors_clean, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
ax.axvline(x=0, color='black', linestyle='-', linewidth=2.5)
ax.axvline(x=np.median(residuals_pct), color='blue', linestyle='--', linewidth=2,
           label=f'Median: {np.median(residuals_pct):.2f}%')

ax.set_xlabel('Percentage Error (%)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Percentage Prediction Error Distribution', fontsize=16, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add MAPE
mape = np.mean(np.abs(residuals_pct))
textstr = f'MAPE: {mape:.2f}%\nMedian: {np.median(residuals_pct):.2f}%'
ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=11,
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

plt.tight_layout()
plt.savefig('figures/07_percentage_error_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 8: Error by Volatility Level
# ============================================================================
print("8. Creating error vs volatility level...")
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(y_test, np.abs(residuals), alpha=0.5, s=30, edgecolors='k', linewidth=0.5)
ax.set_xlabel('Actual Volatility Level', fontsize=12)
ax.set_ylabel('Absolute Error', fontsize=12)
ax.set_title('Prediction Error vs Volatility Level', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(y_test, np.abs(residuals), 1)
p = np.poly1d(z)
sorted_y = np.sort(y_test)
ax.plot(sorted_y, p(sorted_y), "r--", linewidth=2.5, alpha=0.8, 
        label=f'Trend: y={z[0]:.6f}x+{z[1]:.6f}')
ax.legend(fontsize=11)

plt.tight_layout()
plt.savefig('figures/08_error_vs_volatility_level.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 9: Prediction Accuracy by Quartile
# ============================================================================
print("9. Creating accuracy by quartile...")
quartiles = pd.qcut(y_test, q=4, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'])
quartile_mae = [np.mean(np.abs(residuals[quartiles == q])) for q in quartiles.unique()]
quartile_r2 = [r2_score(y_test[quartiles == q], y_pred.flatten()[quartiles == q]) 
               for q in quartiles.unique()]

fig, ax = plt.subplots(figsize=(10, 7))
x_pos = np.arange(len(quartiles.unique()))
bars = ax.bar(x_pos, quartile_mae, alpha=0.7, color='steelblue', edgecolor='black', width=0.6)
ax.set_xlabel('Volatility Quartile', fontsize=12)
ax.set_ylabel('Mean Absolute Error', fontsize=12)
ax.set_title('Prediction Error by Volatility Quartile', fontsize=16, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(quartiles.unique())
ax.grid(True, alpha=0.3, axis='y')

# Add values on bars
for i, (v, r2_val) in enumerate(zip(quartile_mae, quartile_r2)):
    ax.text(i, v + max(quartile_mae)*0.02, f'MAE: {v:.6f}\nR²: {r2_val:.3f}', 
            ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('figures/09_accuracy_by_quartile.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 10: Cumulative Error
# ============================================================================
print("10. Creating cumulative error...")
cumulative_error = np.cumsum(np.abs(residuals))
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(test_dates, cumulative_error, linewidth=2.5, color='darkgreen')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Cumulative Absolute Error', fontsize=12)
ax.set_title('Cumulative Prediction Error', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

# Add slope line
total_error = cumulative_error.iloc[-1] if hasattr(cumulative_error, 'iloc') else cumulative_error[-1]
avg_rate = total_error / len(cumulative_error)
ax.plot(test_dates, np.arange(len(test_dates)) * avg_rate, 'r--', linewidth=2, 
        label=f'Average rate: {avg_rate:.6f}/step')
ax.legend(fontsize=11)

plt.tight_layout()
plt.savefig('figures/10_cumulative_error.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 11: Error Autocorrelation
# ============================================================================
print("11. Creating error autocorrelation...")
fig, ax = plt.subplots(figsize=(12, 6))
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(pd.Series(residuals), ax=ax)
ax.set_title('Residuals Autocorrelation', fontsize=16, fontweight='bold')
ax.set_xlabel('Lag', fontsize=12)
ax.set_ylabel('Autocorrelation', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figures/11_error_autocorrelation.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 12: Prediction Confidence Intervals
# ============================================================================
print("12. Creating prediction with confidence intervals...")
fig, ax = plt.subplots(figsize=(14, 6))

# Plot every Nth point to avoid overcrowding
step = max(1, len(test_dates) // 200)
dates_sample = test_dates[::step]
y_test_sample = y_test[::step]
y_pred_sample = y_pred[::step].flatten()
residuals_sample = residuals[::step]

# Calculate prediction intervals (±2 std)
prediction_interval = 2 * std_residuals

ax.plot(dates_sample, y_test_sample, 'o-', label='Actual', linewidth=2, markersize=4, alpha=0.8)
ax.plot(dates_sample, y_pred_sample, 's-', label='Predicted', linewidth=2, markersize=4, alpha=0.8)
ax.fill_between(dates_sample, 
                y_pred_sample - prediction_interval,
                y_pred_sample + prediction_interval,
                alpha=0.2, color='gray', label='95% Prediction Interval')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Volatility', fontsize=12)
ax.set_title('Predictions with 95% Confidence Intervals', fontsize=16, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('figures/12_predictions_with_confidence_intervals.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Print detailed statistics
# ============================================================================
print("\n" + "="*70)
print("DETAILED MODEL PERFORMANCE STATISTICS")
print("="*70)
print(f"\nRegression Metrics:")
print(f"  R² Score:                    {r2:.6f}")
print(f"  Mean Squared Error (MSE):    {mse:.6f}")
print(f"  Root Mean Squared Error:     {rmse:.6f}")
print(f"  Mean Absolute Error (MAE):   {mae:.6f}")
print(f"  Mean Absolute % Error:       {mape:.2f}%")

print(f"\nResidual Statistics:")
print(f"  Mean:                        {residuals.mean():.6f}")
print(f"  Median:                      {np.median(residuals):.6f}")
print(f"  Std Deviation:               {residuals.std():.6f}")
print(f"  Min Error:                   {residuals.min():.6f}")
print(f"  Max Error:                   {residuals.max():.6f}")
print(f"  Skewness:                    {skew:.4f}")
print(f"  Kurtosis:                    {kurt:.4f}")

print(f"\nError Distribution:")
print(f"  % predictions within 1 MAE:  {np.sum(np.abs(residuals) <= mae) / len(residuals) * 100:.2f}%")
print(f"  % predictions within 2 MAE:  {np.sum(np.abs(residuals) <= 2*mae) / len(residuals) * 100:.2f}%")
print(f"  % predictions within 1σ:     {np.sum(np.abs(residuals) <= std_residuals) / len(residuals) * 100:.2f}%")
print(f"  % predictions within 2σ:     {np.sum(np.abs(residuals) <= 2*std_residuals) / len(residuals) * 100:.2f}%")

print(f"\nPrediction Quality by Quartile:")
for i, q in enumerate(quartiles.unique()):
    print(f"  {q}: MAE = {quartile_mae[i]:.6f}, R² = {quartile_r2[i]:.4f}")

print("\n" + "="*70)
print("ALL PLOTS SAVED SUCCESSFULLY!")
print("="*70)
print("\nGenerated files in 'figures/' directory:")
for i in range(1, 13):
    print(f"  {i:02d}_*.png")
print("="*70)

GENERATING INDIVIDUAL PLOTS...

1. Creating Actual vs Predicted timeline...
2. Creating scatter plot...
3. Creating residuals timeline...
4. Creating residuals distribution...
5. Creating Q-Q plot...
6. Creating absolute error timeline...
7. Creating percentage error distribution...
8. Creating error vs volatility level...
9. Creating accuracy by quartile...
10. Creating cumulative error...
11. Creating error autocorrelation...
12. Creating prediction with confidence intervals...

DETAILED MODEL PERFORMANCE STATISTICS

Regression Metrics:
  R² Score:                    0.581421
  Mean Squared Error (MSE):    0.000025
  Root Mean Squared Error:     0.005018
  Mean Absolute Error (MAE):   0.003211
  Mean Absolute % Error:       40.45%

Residual Statistics:
  Mean:                        0.000144
  Median:                      -0.000628
  Std Deviation:               0.005016
  Min Error:                   -0.012425
  Max Error:                   0.050183
  Skewness:                    3.

## Directional Accuracy Analysis

### Purpose
Evaluate the model's ability to predict the direction of volatility changes (increase vs. decrease), not just the magnitude. This is critical for trading applications where knowing which way volatility will move matters more than the exact value.

### Key Insight: Direction vs. Magnitude

While the model achieves strong R² (~57%) for predicting volatility **levels**, directional accuracy is a separate challenge:
- **High R²** = Good at predicting "how much" volatility
- **Directional accuracy** = Good at predicting "which way" volatility moves

A model can have high R² but poor directional accuracy if it correctly estimates magnitudes but frequently misses turning points.

### Three Evaluation Methods

**Method 1: Sequential Forecast Changes**
- Compares consecutive 10-day forecasts: "Will tomorrow's forecast be higher than today's?"
- Accounts for the 9-day overlap in predictions (day 1-10 vs. day 2-11)
- Measures incremental forecast revision accuracy
- Most relevant for daily forecast updates

**Method 2: Binary Classification (Up/Down)**
- Treats direction prediction as a classification problem
- **Confusion Matrix** shows:
  - True Positives: Correctly predicted increases
  - True Negatives: Correctly predicted decreases
  - False Positives: Predicted increase, actually decreased (Type I error)
  - False Negatives: Predicted decrease, actually increased (Type II error)
- Includes precision, recall, and F1-scores for each direction
- Benchmark: 50% (random coin flip)

**Method 3: Three-Class (Up/Down/Flat)**
- Adds "flat" category for very small changes (<1% of volatility std dev)
- More realistic: not every change is meaningful
- Harder task but more useful for practical applications
- Avoids penalizing the model for missing noise

### Why Directional Accuracy Might Be Low

**Overlapping Windows**: 10-day forecasts share 9 days, so consecutive changes are small and noisy

**Temporal Lag**: LSTMs often predict values close to the last observation, causing lag at turning points

**Regime Changes**: Volatility direction shifts are inherently difficult to predict—even humans struggle

**Optimization Mismatch**: Model was trained to minimize MSE (magnitude error), not maximize directional accuracy

### Diagnostic Plots

**1. Confusion Matrix Heatmap**
- Visual breakdown of correct/incorrect directional predictions
- Shows if model is biased (e.g., over-predicting increases)

**2. Accuracy by Change Magnitude**
- Splits predictions into small/medium/large changes
- Tests hypothesis: "Does model perform better on significant moves?"
- Shows sample size for each category

**3. Actual vs. Predicted Changes Scatter**
- Quadrant analysis:
  - Green quadrants (top-right, bottom-left) = correct direction
  - Red quadrants (top-left, bottom-right) = wrong direction
- Distance from perfect prediction line shows magnitude error
- Clustering near axes = model hesitant to predict large changes

**4. Rolling Directional Accuracy**
- 50-period moving average smooths short-term noise
- Identifies periods where model performs better/worse
- Compares to random chance (50%) and overall average

**5. Change Magnitude Distribution**
- Overlapping histograms: correct vs. incorrect predictions
- Tests if errors occur more on small or large changes
- Mean comparison shows if model struggles with certain volatility regimes

### Interpretation Guidelines

**Good Directional Performance:**
- Accuracy consistently >55% (significant edge over random)
- Balanced precision/recall for both increases and decreases
- Better accuracy on large changes (meaningful predictions)
- Stable rolling accuracy (not regime-dependent)

**Warning Signs:**
- ~50% accuracy = model not capturing directional information
- High bias (predicting one direction far more than it occurs)
- Worse on large changes = missing the most important signals
- Deteriorating rolling accuracy = concept drift over time

### Practical Implications

**For Trading:**
- Directional accuracy >55% can be profitable with proper risk management
- Even 52-53% edge is valuable for position sizing decisions
- Combine with magnitude predictions for optimal entry/exit timing

**For Risk Management:**
- Focus on magnitude predictions (where model excels)
- Use directional signal with caution or as supplementary information
- Consider regime-based strategies (model performs better in certain volatility regimes)

### Alternative Approaches

If directional accuracy is critical:
1. **Train separate classifier** specifically for direction using binary cross-entropy loss
2. **Add directional loss term** to training objective (multi-task learning)
3. **Use non-overlapping windows** (every 10th prediction) for cleaner signals
4. **Engineer momentum features** (RSI, MACD) to capture trend information

In [8]:
### Directional volatility change analysis and plots

# Load the best model
best_model = load_model('best_lstm_model.keras')

# Prepare test data
X_test_reshaped = X_test.values.reshape(-1, X_test.shape[1], 1)

# Get predictions
y_pred = best_model.predict(X_test_reshaped).flatten()

# Convert to actual values
y_true = y_test.values

print("="*70)
print("DIRECTIONAL ACCURACY ANALYSIS")
print("="*70)

# Calculate the direction (sign) of changes
# For the first prediction, we compare against the last training value
# For subsequent predictions, we compare against the previous actual value

# Method 1: Direction relative to previous actual value
if len(y_true) > 1:
    # Calculate actual changes (direction)
    actual_changes = np.diff(y_true)
    actual_direction = np.sign(actual_changes)  # 1 for increase, -1 for decrease, 0 for no change
    
    # Calculate predicted changes
    # Compare prediction to previous actual value
    predicted_changes = y_pred[1:] - y_true[:-1]
    predicted_direction = np.sign(predicted_changes)
    
    # Calculate directional accuracy
    directional_matches = (actual_direction == predicted_direction)
    directional_accuracy = np.mean(directional_matches) * 100
    
    print(f"\nMethod 1: Direction relative to previous actual value")
    print(f"Directional Accuracy: {directional_accuracy:.2f}%")
    print(f"Correctly predicted direction: {np.sum(directional_matches)} out of {len(directional_matches)}")
    
    # Separate analysis for increases vs decreases
    increase_mask = actual_direction > 0
    decrease_mask = actual_direction < 0
    
    if np.sum(increase_mask) > 0:
        increase_accuracy = np.mean(directional_matches[increase_mask]) * 100
        print(f"Accuracy predicting INCREASES: {increase_accuracy:.2f}% ({np.sum(directional_matches[increase_mask])}/{np.sum(increase_mask)})")
    
    if np.sum(decrease_mask) > 0:
        decrease_accuracy = np.mean(directional_matches[decrease_mask]) * 100
        print(f"Accuracy predicting DECREASES: {decrease_accuracy:.2f}% ({np.sum(directional_matches[decrease_mask])}/{np.sum(decrease_mask)})")

# Method 2: Binary classification - Up or Down
print(f"\n" + "="*70)
print("Method 2: Binary Classification (Increase vs Decrease)")
print("="*70)

# Calculate changes from previous value
actual_changes_full = np.diff(y_true)
predicted_changes_full = y_pred[1:] - y_true[:-1]

# Binary: 1 for increase, 0 for decrease
actual_binary = (actual_changes_full > 0).astype(int)
predicted_binary = (predicted_changes_full > 0).astype(int)

# Calculate metrics
binary_accuracy = accuracy_score(actual_binary, predicted_binary) * 100
print(f"\nBinary Directional Accuracy: {binary_accuracy:.2f}%")

# Confusion Matrix
cm = confusion_matrix(actual_binary, predicted_binary)
print("\nConfusion Matrix:")
print("                 Predicted Down  Predicted Up")
print(f"Actual Down          {cm[0,0]:6d}        {cm[0,1]:6d}")
print(f"Actual Up            {cm[1,0]:6d}        {cm[1,1]:6d}")

# Classification Report
print("\nClassification Report:")
print(classification_report(actual_binary, predicted_binary, 
                          target_names=['Decrease', 'Increase'],
                          digits=4))

# Method 3: Three-class classification (Up, Down, Flat)
print(f"\n" + "="*70)
print("Method 3: Three-Class Classification (Increase/Decrease/Flat)")
print("="*70)

# Define threshold for "flat" (e.g., changes less than 1% of std)
threshold = 0.01 * np.std(y_true)

# Three-class: 1 for increase, -1 for decrease, 0 for flat
actual_three_class = np.sign(actual_changes_full)
actual_three_class[np.abs(actual_changes_full) < threshold] = 0

predicted_three_class = np.sign(predicted_changes_full)
predicted_three_class[np.abs(predicted_changes_full) < threshold] = 0

three_class_accuracy = np.mean(actual_three_class == predicted_three_class) * 100
print(f"\nThree-Class Accuracy: {three_class_accuracy:.2f}%")

# Detailed breakdown
for cls, name in [(1, 'Increases'), (-1, 'Decreases'), (0, 'Flat')]:
    mask = actual_three_class == cls
    if np.sum(mask) > 0:
        cls_accuracy = np.mean((actual_three_class == predicted_three_class)[mask]) * 100
        print(f"{name}: {cls_accuracy:.2f}% ({np.sum((actual_three_class == predicted_three_class)[mask])}/{np.sum(mask)})")

# Visualization
print(f"\n" + "="*70)
print("GENERATING VISUALIZATIONS...")
print("="*70)

# Split by change magnitude
change_magnitudes = np.abs(actual_changes)
small_threshold = np.percentile(change_magnitudes, 33)
large_threshold = np.percentile(change_magnitudes, 67)

small_moves = change_magnitudes < small_threshold
medium_moves = (change_magnitudes >= small_threshold) & (change_magnitudes < large_threshold)
large_moves = change_magnitudes >= large_threshold

# ============================================================================
# Plot 1: Confusion Matrix Heatmap
# ============================================================================
print("\n1. Creating confusion matrix heatmap...")
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Down', 'Up'], yticklabels=['Down', 'Up'],
            cbar_kws={'label': 'Count'}, annot_kws={'size': 16})
ax.set_title('Confusion Matrix: Directional Predictions', fontsize=16, fontweight='bold', pad=20)
ax.set_ylabel('Actual Direction', fontsize=13)
ax.set_xlabel('Predicted Direction', fontsize=13)

# Add accuracy text
textstr = f'Overall Accuracy: {binary_accuracy:.2f}%'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
ax.text(0.98, 0.02, textstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='bottom', horizontalalignment='right', bbox=props)

plt.tight_layout()
plt.savefig('figures/directional_01_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 2: Directional Accuracy by Change Magnitude
# ============================================================================
print("2. Creating accuracy by change magnitude...")
fig, ax = plt.subplots(figsize=(10, 8))
accuracies = [np.mean(directional_matches[small_moves])*100,
              np.mean(directional_matches[medium_moves])*100,
              np.mean(directional_matches[large_moves])*100]
labels = ['Small\nChanges', 'Medium\nChanges', 'Large\nChanges']
colors = ['lightblue', 'skyblue', 'steelblue']

bars = ax.bar(labels, accuracies, color=colors, edgecolor='black', alpha=0.8, width=0.6)
ax.axhline(y=50, color='red', linestyle='--', linewidth=2, label='Random chance (50%)')
ax.set_ylabel('Directional Accuracy (%)', fontsize=12)
ax.set_title('Directional Accuracy by Change Magnitude', fontsize=16, fontweight='bold', pad=20)
ax.set_ylim([0, 100])
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 2,
            f'{acc:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

# Add count labels
counts = [np.sum(small_moves), np.sum(medium_moves), np.sum(large_moves)]
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2., 5,
            f'n={count}', ha='center', va='bottom', fontsize=10, style='italic')

plt.tight_layout()
plt.savefig('figures/directional_02_accuracy_by_magnitude.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 3: Actual vs Predicted Changes
# ============================================================================
print("3. Creating actual vs predicted changes scatter...")
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(actual_changes_full, predicted_changes_full, alpha=0.5, s=30, edgecolors='k', linewidth=0.5)
ax.axhline(y=0, color='k', linestyle='-', linewidth=1)
ax.axvline(x=0, color='k', linestyle='-', linewidth=1)

# Perfect prediction line
max_abs = max(abs(actual_changes_full.max()), abs(actual_changes_full.min()),
              abs(predicted_changes_full.max()), abs(predicted_changes_full.min()))
ax.plot([-max_abs, max_abs], [-max_abs, max_abs], 'r--', linewidth=2.5, label='Perfect prediction')

ax.set_title('Actual vs Predicted Changes', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Actual Change in Volatility', fontsize=12)
ax.set_ylabel('Predicted Change in Volatility', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Highlight quadrants
ax.fill_between([0, max_abs], 0, max_abs, alpha=0.1, color='green', label='_nolegend_')  # Top-right
ax.fill_between([-max_abs, 0], -max_abs, 0, alpha=0.1, color='green', label='_nolegend_')  # Bottom-left
ax.fill_between([0, max_abs], -max_abs, 0, alpha=0.1, color='red', label='_nolegend_')  # Bottom-right
ax.fill_between([-max_abs, 0], 0, max_abs, alpha=0.1, color='red', label='_nolegend_')  # Top-left

# Add quadrant labels
ax.text(0.7, 0.95, 'Correct\nDirection', transform=ax.transAxes, fontsize=10,
        verticalalignment='top', horizontalalignment='center',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
ax.text(0.3, 0.05, 'Correct\nDirection', transform=ax.transAxes, fontsize=10,
        verticalalignment='bottom', horizontalalignment='center',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

plt.tight_layout()
plt.savefig('figures/directional_03_changes_scatter.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 4: Rolling Directional Accuracy
# ============================================================================
print("4. Creating rolling directional accuracy...")
fig, ax = plt.subplots(figsize=(14, 6))
window = 50
rolling_accuracy = pd.Series(directional_matches.astype(int)).rolling(window=window).mean() * 100
ax.plot(rolling_accuracy, linewidth=2.5, color='steelblue')
ax.axhline(y=50, color='red', linestyle='--', linewidth=2, label='Random chance (50%)')
ax.axhline(y=directional_accuracy, color='green', linestyle='--', linewidth=2,
           label=f'Overall accuracy ({directional_accuracy:.1f}%)')
ax.set_title(f'Rolling Directional Accuracy (window={window})', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Time Step', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 100])

plt.tight_layout()
plt.savefig('figures/directional_04_rolling_accuracy.png', dpi=300, bbox_inches='tight')
plt.close()

# ============================================================================
# Plot 5: Change Magnitude Distribution (Correct vs Incorrect)
# ============================================================================
print("5. Creating change magnitude distribution...")
fig, ax = plt.subplots(figsize=(12, 6))
correct_direction = np.abs(actual_changes[directional_matches])
incorrect_direction = np.abs(actual_changes[~directional_matches])

ax.hist([correct_direction, incorrect_direction], 
        bins=40, label=['Correct Direction', 'Incorrect Direction'],
        alpha=0.7, edgecolor='black', color=['green', 'red'])
ax.set_xlabel('Magnitude of Actual Change', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Change Magnitude: Correct vs Incorrect Predictions', fontsize=16, fontweight='bold', pad=20)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

# Add statistics
textstr = f'Correct mean: {np.mean(correct_direction):.6f}\nIncorrect mean: {np.mean(incorrect_direction):.6f}'
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
ax.text(0.98, 0.97, textstr, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', horizontalalignment='right', bbox=props)

plt.tight_layout()
plt.savefig('figures/directional_05_magnitude_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# Summary Statistics
print(f"\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)
print(f"Total predictions analyzed: {len(directional_matches)}")
print(f"Overall directional accuracy: {directional_accuracy:.2f}%")
print(f"Better than random (50%): {'YES' if directional_accuracy > 50 else 'NO'}")
print(f"Improvement over random: {directional_accuracy - 50:.2f} percentage points")
print(f"\nActual volatility increases: {np.sum(actual_binary)} ({np.sum(actual_binary)/len(actual_binary)*100:.1f}%)")
print(f"Actual volatility decreases: {np.sum(1-actual_binary)} ({np.sum(1-actual_binary)/len(actual_binary)*100:.1f}%)")
print(f"Predicted increases: {np.sum(predicted_binary)} ({np.sum(predicted_binary)/len(predicted_binary)*100:.1f}%)")
print(f"Predicted decreases: {np.sum(1-predicted_binary)} ({np.sum(1-predicted_binary)/len(predicted_binary)*100:.1f}%)")
print("="*70)

print("\n" + "="*70)
print("ALL DIRECTIONAL ACCURACY PLOTS SAVED!")
print("="*70)
print("\nGenerated files in 'figures/' directory:")
print("  directional_01_confusion_matrix.png")
print("  directional_02_accuracy_by_magnitude.png")
print("  directional_03_changes_scatter.png")
print("  directional_04_rolling_accuracy.png")
print("  directional_05_magnitude_distribution.png")
print("="*70)

146/146 [==============================] - 0s 1ms/step
DIRECTIONAL ACCURACY ANALYSIS

Method 1: Direction relative to previous actual value
Directional Accuracy: 52.18%
Correctly predicted direction: 2431 out of 4659
Accuracy predicting INCREASES: 60.25% (1396/2317)
Accuracy predicting DECREASES: 44.19% (1035/2342)

Method 2: Binary Classification (Increase vs Decrease)

Binary Directional Accuracy: 52.18%

Confusion Matrix:
                 Predicted Down  Predicted Up
Actual Down            1035          1307
Actual Up               921          1396

Classification Report:
              precision    recall  f1-score   support

    Decrease     0.5291    0.4419    0.4816      2342
    Increase     0.5165    0.6025    0.5562      2317

    accuracy                         0.5218      4659
   macro avg     0.5228    0.5222    0.5189      4659
weighted avg     0.5228    0.5218    0.5187      4659


Method 3: Three-Class Classification (Increase/Decrease/Flat)

Three-Class Accuracy: 41.5